### Conv2d 적용하기
* Conv2d Layer의 주요 생성 인자
    * in_channels: 입력 tensor의 channel 수(차원)
    * out_channels: conv 연산 적용 후 생성되는 출력 tensor(output feature map)의 차원 수
    * kernel_size: conv kernel size. (5, 5)와 같은 튜플 형태(이 경우 5x5 kernel size) 또는 5와 같이 정수값
    * stride: conv연산 stride
    * padding: conv 연산 전 입력 데이터의 상하좌우로 채우는 빈 값 size(output feature map의 사이즈 크기 조정을 위해 사용)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

input_tensor = torch.randn(3, 28, 28)
print('input tensor shape:', input_tensor.shape)

conv_layer_01 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5, stride=1)
output_tensor = conv_layer_01(input_tensor)
print('output tensor shape:', output_tensor.shape)

In [ ]:
conv_layer_01 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5, stride=1, padding=0)

In [ ]:
#conv2d layer weight shape
conv_layer_01.weight.shape

In [ ]:
input_tensor = torch.randn(3, 28, 28)

# 2개의 convolution을 적용하여 최종 output의 shape가 (16, 22, 22)가 나옴. 
conv_layer_01 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=5, stride=1) #padding='same'
conv_layer_02 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1) #padding='same'
output_01 = conv_layer_01(input_tensor)
output_02 = conv_layer_02(output_01)

print('output_01 shape:', output_01.shape, 'output_02 shape:', output_02.shape)

In [ ]:
conv_layer_01.weight.shape, conv_layer_02.weight.shape

### MaxPool2d(AvgPool2d) 적용
* MaxPool2d은 kernel_size 만큼 window 이동으로 가장 큰 값을 추출해 가면 output을 생성.
* Pooling은 학습 파라미터를 가지고 있지 않은 Layer
* 주요 기능
  * 입력 feature map의 사이즈를 줄여서 computational cost 감소
  * 차원 축소의 역할로서 특정 영역별로(kernel_size) 입력 feature map을 주요한 feature값으로 요약
  * 입력값이 작은 변화에 너무 민감하게 반응하지 않도록 변동성 감소(overfitting 감소)
* kernel_size, stride, padding을 생성 파라미터로 가짐. stride의 default값이 kernel_size 값임에 유의

In [ ]:
input_tensor = torch.randn(3, 28, 28)

conv_layer_01 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
pool_layer_01 = nn.MaxPool2d(kernel_size=2)# stride는 기재하지 않으면 kernel_size와 동일. 
output_01 = conv_layer_01(input_tensor)
output_02 = pool_layer_01(output_01)

print('output_01 shape:', output_01.shape, 'output_02 shape:', output_02.shape)


### CNN 기반 모델 생성 - 01
* Conv2d 기반으로 모델 생성. Conv2d -> ReLU -> Conv2d -> ReLU -> MaxPool2d 로 CNN 모델 생성
* 마지막 classification layer는 Linear Layer가 되어야 하므로 Conv2d 수행 결과인 3차원 Feature Map을 flatten하여 Linear Layer 연결 필요. 이를 위해 Flatten을 Feature map에 적용한 뒤 Linear Layer로 연결
* 마지막 Feature Map -> Flatten -> Linear 적용을 하게 되면 Linear에 매우 많은 Learnable Parameter 를 가지게 됨(Overfitting의 이슈 발생하기 쉬움. Drop out등의 설정 필요 할 수 있음)
* Linear Layer의 입력 in_features는 Flatten의 결과로 만들어진 입력의 갯수를 설정해줘야 하지만, 이미지 크기나 Conv 설정에 따라 변하게 됨.이를 위해 최종 feature map의 크기를 식으로 계산하거나, summary등을 통해 미리 파악 후 적용 필요. 하지만 이미지 크기나 Conv 설정이 변경되면 다시 계산해야 하는 불편함이 있음.
  

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary

NUM_INPUT_CHANNELS = 3

# 3x3 kernel, 32개의 filter들을 가지는 Conv Layer, 3x3 kernel, 64개의 filter들을 가지는 Conv Layer, 이후 MaxPooling 
class SimpleCNN_01(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv_1 = nn.Conv2d(in_channels=NUM_INPUT_CHANNELS, out_channels=32, kernel_size=3, stride=1)
        self.conv_2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.flatten = nn.Flatten()
        self.classifier = nn.Linear(in_features=12544, out_features=num_classes)

    def forward(self, x):
        x = F.relu(self.conv_1(x))
        x = F.relu(self.conv_2(x))
        x = self.pool(x)
        x = self.flatten(x)
        x = self.classifier(x)

        return x

input = torch.randn(1, 3, 32, 32) # 이미지 사이즈를 64, 64로 변경하면 classfication layer 오류 발생.
simple_cnn_01 = SimpleCNN_01(num_classes=10)
output = simple_cnn_01(input)
print(output.shape)

In [ ]:
from torchinfo import summary

summary(model=simple_cnn_01, input_size=(1, 3, 32, 32),
        col_names=['input_size', 'output_size', 'num_params'],
        row_settings=['var_names'])


### AdaptiveAvgPool2d를 이용한 Global Average Pooling
* Pytorch에서는 GAP를 위해서 AdaptiveAvgPool2d()를 적용
* AdaptiveAvgPool2d는 인자로 output_size를 받으며, channel별로 자동으로 지정된 output_size가 되도록 subsampling 수행. 

In [ ]:
input = torch.randn(1, 64, 8, 9)

# 만들어지는 (채널별)출력 size는 5x5
m = nn.AdaptiveAvgPool2d(output_size=(5, 5))
output = m(input)
print(output.shape)

m = nn.AdaptiveAvgPool2d(output_size=(1, 1))
output = m(input)
print(output.shape)


### CNN 기반 모델 생성 - 02
* Feature map을 바로 Flatten하지 않고 Adaptive Global Pooling을 적용한 뒤 Flatten 적용
* Global Pooling은 MaxPool2d와 다르게 채널별로 하나의 값으로 Pooling을 적용할 수 있음. 보통은 AdaptiveAvgPool2d가 많이 활용됨.
* 마지막 Feature Map에 AdaptiveAvgPool2d(output_size=(1, 1))을 적용하면 feature map의 채널수는 동일하지만 면적(가로와 세로)은 1인 feature map으로 Pooling됨. 때문에 마지막 Conv2d의 out_channels 수만 알면 Flatten을 생기는 차원을 알 수 있으며, 이를 Linear Layer의 in_features의 값으로 입력하면 됨.
* Global Pooling은 Feature Map 압축 효과로 인하여 classification Layer의 파라미터를 크게 줄일 수 있음.
* Global Pooling은 보통 CNN의 Layer가 어느정도 깊이가 있어야 성능 저하가 발생하지 않음. 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary

NUM_INPUT_CHANNELS = 3

class SimpleCNN_02(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.conv_1 = nn.Conv2d(in_channels=NUM_INPUT_CHANNELS, out_channels=32, kernel_size=3, stride=1)
        self.conv_2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1)
        self.pool = nn.MaxPool2d(kernel_size=2)
        self.adapt_pool = nn.AdaptiveAvgPool2d(output_size=(1, 1))
                
        # in_features는 마지막 Conv2d의 out_channels임.
        self.classifier = nn.Linear(in_features=64, out_features=num_classes)

    def forward(self, x):
        x = F.relu(self.conv_1(x))
        x = F.relu(self.conv_2(x))
        x = self.pool(x)

        # Global Pooling 적용. 
        x = self.adapt_pool(x)
        x = x.view(x.size(0), -1) #x = torch.flatten(x, start_dim=1)
        x = self.classifier(x)

        return x
        
input = torch.randn(1, 3, 64, 64) # 이미지 사이즈를 64, 64로 변경해도 classfication layer 오류 없음.
simple_cnn_02 = SimpleCNN_02(num_classes=10)
output = simple_cnn_02(input)
print(output.shape)

In [ ]:
simple_cnn_02 = SimpleCNN_02(num_classes=10)
summary(model=simple_cnn_02, input_size=(1, 3, 32, 32), # 이미지 사이즈를 64, 64로 변경해도 오류 발생하지 않음.  
        col_names=['input_size', 'output_size', 'num_params'], 
        row_settings=['var_names'])

In [ ]:
import torchvision.models as models

# torchvision의 pretrained 모델 구조에서 Global Average Pooling 적용 살펴 보기
model = models.vgg19() # models.resnet50()
print(model)

### CIFAR10 Dataset 생성
* torchvision.datasets의 CIFAR10으로 dataset 생성. transform=ToTensor() 수행 시 PIL image를 tensor로 변환하면서 0 ~ 1사이 값으로 Normalization 적용.
* CIFAR10 Dataset의 data 속성은 numpy 형태로 이미지값을 가짐. targets 속성은 np.uint8 형태로 target값을 가짐. classes 속성은 개별 target에 매핑되는 class의 이름을 가짐.  
* CIFAR10 Dataset 기반으로 DataLoader 생성.

In [ ]:
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from torch.utils.data import random_split

#전체 6만개 데이터 중, 5만개는 학습 데이터용. 이를 다시 학습과 검증용으로 split , 1만개는 테스트 데이터용
train_dataset = CIFAR10(root='./data', train=True, download=True, transform=ToTensor())
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=ToTensor())

tr_size = int(0.85 * len(train_dataset))
val_size = len(train_dataset) - tr_size
tr_dataset, val_dataset = random_split(train_dataset, [tr_size, val_size])
print('tr:', len(tr_dataset), 'valid:', len(val_dataset))

tr_loader = DataLoader(tr_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

In [ ]:
images, labels = next(iter(tr_loader))
print(images.shape, labels.shape)
print(images[0].max(), images[0].min(), labels.min(), labels.max())

In [ ]:
# tr_dataset은 Subset
print(type(tr_dataset), '\n', type(train_dataset))
print(tr_dataset, '\n', train_dataset)

In [ ]:
# targets는 10개 target classes의 0 ~ 9까지의 값. clsses는 0~9까지의 target값에 매핑되는 label명
# tr_dataset은 subset으로 .classes 속성이 없음. train_dataset.classes로 확인
print(train_dataset.classes)

In [ ]:
train_dataset.data

In [ ]:
#train_dataset[0]은 호출시 마다 ToTensor()로 변환됨. 이미지 표현을 위해 PIL이나 Numpy array 필요. 
#train_dataset.data 는 image를 numpy 배열 형태로 가짐(channel last)
print(type(train_dataset.data), train_dataset.data.shape)

In [ ]:
import matplotlib.pyplot as plt

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

def show_images(images, labels, ncols=8):
    figure, axs = plt.subplots(figsize=(22, 6), nrows=1, ncols=ncols)
    for i in range(ncols):
        # imshow()는 numpy array를 그대로 이미지화 시킬 수 있음. 
        axs[i].imshow(images[i])
        axs[i].set_title(class_names[labels[i]])
        
show_images(train_dataset.data[:8], train_dataset.targets[:8], ncols=8)
show_images(train_dataset.data[8:16], train_dataset.targets[8:16], ncols=8)

### CNN 기반 모델 생성 - 03
* Conv의 필터수 및 네트웍의 깊이를 좀 더 증가 시켜 모델 구성.
* conv->relu->conv->relu->pooling을 연속적으로 수행. kernel 크기는 3, filter 수는 블럭별로 32 -> 64 -> 128로 증가. 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary

NUM_INPUT_CHANNELS = 3

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        #padding 1로 conv 적용 후 출력 면적 사이즈를 입력 면적 사이즈와 동일하게 유지.
        #kernel 크기 3, filter 개수 32 연속 적용.
        self.conv_11 = nn.Conv2d(in_channels=NUM_INPUT_CHANNELS, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv_12 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool_01 = nn.MaxPool2d(kernel_size=2)
        
        #out_channels이 64인 2개의 Conv2d 연속 적용. stride=1이 기본값, padding='same'은 version 1.8에서 소개됨.  
        self.conv_21 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding='same')
        self.conv_22 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding='same')
        self.pool_02 = nn.MaxPool2d(kernel_size=2)

        # Sequential Module을 이용하여 Conv Layer들을 생성. 이 경우 relu activation위해 ReLU Layer 연결 생성 필요.
        # filter갯수 128개인 Conv Layer 2개 적용 후 Max Pooling 적용.
        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        # GAP 및 최종 Classifier Layer
        self.adapt_pool = nn.AdaptiveAvgPool2d(output_size=(1,1))
        self.classifier = nn.Linear(in_features=128, out_features=num_classes)
        # self.classifier_block = nn.Sequential(
        #     nn.AdaptiveAvgPool2d(output_size=(1, 1)),
        #     nn.Flatten(),
        #     nn.Linear(in_features=128, out_features=num_classes)
        # )
        
    def forward(self, x):
        x = F.relu(self.conv_11(x))
        x = F.relu(self.conv_12(x))
        x = self.pool_01(x)
        # x = F.max_pool2d(x, 2)

        x = F.relu(self.conv_21(x))
        x = F.relu(self.conv_22(x))
        x = self.pool_02(x)

        x = self.conv_block(x)
        
        # global pooling
        x = self.adapt_pool(x)
        x = torch.flatten(x, start_dim=1) #또는 x = x.view(x.size(0), -1)
        
        # final classification
        x = self.classifier(x)
        # 또는 아래와 같이 classifier_block을 forward
        
        return x


simple_cnn = SimpleCNN(num_classes=10)

summary(model=simple_cnn, input_size=(1, 3, 32, 32), 
        col_names=['input_size', 'output_size', 'num_params'], 
        row_settings=['var_names'])


### Trainer 클래스를 이용하여 학습 수행

In [ ]:
from tqdm import tqdm
import torch.nn.functional as F

class Trainer:
    def __init__(self, model, loss_fn, optimizer, train_loader, val_loader, device=None):
        self.model = model.to(device)
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = device
    
    def train_epoch(self, epoch):
        self.model.train()
        
        # running 평균 loss 계산. 
        accu_loss = 0.0
        running_avg_loss = 0.0
        # 정확도, 정확도 계산을 위한 전체 건수 및 누적 정확건수
        num_total = 0.0
        accu_num_correct = 0.0
        accuracy = 0.0
        
        # tqdm으로 실시간 training loop 진행 상황 시각화
        with tqdm(total=len(self.train_loader), desc=f"Epoch {epoch+1} [Training..]", leave=True) as progress_bar:
            for batch_idx, (inputs, targets) in enumerate(self.train_loader):
                # 반드시 to(self.device). to(device) 아님. 
                inputs = inputs.to(self.device)
                targets = targets.to(self.device)
                
                # Forward pass
                outputs = self.model(inputs)
                loss = self.loss_fn(outputs, targets)
                
                # Backward pass
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

                # batch 반복 시 마다 누적  loss를 구하고 이를 batch 횟수로 나눠서 running 평균 loss 구함.  
                accu_loss += loss.item()
                running_avg_loss = accu_loss /(batch_idx + 1)

                # accuracy metric 계산
                # outputs 출력 예측 class값과 targets값 일치 건수 구하고
                num_correct = (outputs.argmax(-1) == targets).sum().item()
                # 배치별 누적 전체 건수와 누적 전체 num_correct 건수로 accuracy 계산
                num_total += inputs.shape[0]
                accu_num_correct += num_correct
                accuracy = accu_num_correct / num_total

                #tqdm progress_bar에 진행 상황 및 running 평균 loss와 정확도 표시
                progress_bar.update(1)
                if batch_idx % 20 == 0 or (batch_idx + 1) == progress_bar.total:  # 20 batch횟수마다 또는 맨 마지막 batch에서 update 
                    progress_bar.set_postfix({"Loss": running_avg_loss, 
                                              "Accuracy": accuracy})
        
        return running_avg_loss, accuracy
                
    def validate_epoch(self, epoch):
        if not self.val_loader:
            return None
            
        self.model.eval()

        # running 평균 loss 계산. 
        accu_loss = 0
        running_avg_loss = 0
        # 정확도, 정확도 계산을 위한 전체 건수 및 누적 정확건수
        num_total = 0.0
        accu_num_correct = 0.0
        accuracy = 0.0
        with tqdm(total=len(self.val_loader), desc=f"Epoch {epoch+1} [Validating]", leave=True) as progress_bar:
            with torch.no_grad():
                for batch_idx, (inputs, targets) in enumerate(self.val_loader):
                    inputs = inputs.to(self.device)
                    targets = targets.to(self.device)
                    
                    outputs = self.model(inputs)
                    
                    loss = self.loss_fn(outputs, targets)
                    # batch 반복 시 마다 누적  loss를 구하고 이를 batch 횟수로 나눠서 running 평균 loss 구함.  
                    accu_loss += loss.item()
                    running_avg_loss = accu_loss /(batch_idx + 1)

                    # accuracy metric 계산
                    # outputs 출력 예측 class값과 targets값 일치 건수 구하고
                    num_correct = (outputs.argmax(-1) == targets).sum().item()
                    # 배치별 누적 전체 건수와 누적 전체 num_correct 건수로 accuracy 계산  
                    num_total += inputs.shape[0]
                    accu_num_correct += num_correct
                    accuracy = accu_num_correct / num_total

                    #tqdm progress_bar에 진행 상황 및 running 평균 loss와 정확도 표시
                    progress_bar.update(1)
                    if batch_idx % 20 == 0 or (batch_idx + 1) == progress_bar.total:  # 20 batch횟수마다 또는 맨 마지막 batch에서 update 
                        progress_bar.set_postfix({"Loss": running_avg_loss, 
                                                  "Accuracy":accuracy})
        return running_avg_loss, accuracy
    
    def fit(self, epochs):
        # epoch 시마다 학습/검증 결과를 기록하는 history dict 생성.
        history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
        for epoch in range(epochs):
            train_loss, train_acc = self.train_epoch(epoch)
            val_loss, val_acc = self.validate_epoch(epoch)
            print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f} Train Accuracy: {train_acc:.4f}",
                  f", Val Loss: {val_loss:.4f} Val Accuracy: {val_acc:.4f}" if val_loss is not None else "")
            # epoch 시마다 학습/검증 결과를 기록.
            history['train_loss'].append(train_loss); history['train_acc'].append(train_acc)
            history['val_loss'].append(val_loss); history['val_acc'].append(val_acc)
            
        return history 
    
    # 학습이 완료된 모델을 return 
    def get_trained_model(self):
        return self.model

In [ ]:
import torch 
import torch.nn as nn
from torch.optim import SGD, Adam

NUM_INPUT_CHANNELS = 3
NUM_CLASSES = 10

model = SimpleCNN(num_classes=NUM_CLASSES)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

trainer = Trainer(model=model, loss_fn=loss_fn, optimizer=optimizer,
       train_loader=tr_loader, val_loader=val_loader, device=device)
# 학습 및 평가 
history = trainer.fit(30)

In [ ]:
import matplotlib.pyplot as plt

# 시각화 
def show_history(history, metric='acc'):
    if metric == 'loss':
        train_metric_name = 'train_loss'
        val_metric_name = 'val_loss'
    else:
        train_metric_name = 'train_acc'
        val_metric_name = 'val_acc'
        
    plt.plot(history[train_metric_name], label='train')
    plt.plot(history[val_metric_name], label='valid')
    plt.legend()
    
show_history(history, metric='loss')

### Predictor 클래스로 모델 성능 평가 및 이미지 예측

In [ ]:
class Predictor:
    def __init__(self, model, device):
        self.model = model.to(device)
        self.device = device

    def evaluate(self, loader):
        self.model.eval()
        eval_metric = 0.0
        
        num_total = 0.0
        accu_num_correct = 0.0

        with tqdm(total=len(loader), desc=f"[Evaluating]", leave=True) as progress_bar:
            with torch.no_grad():
                for batch_idx, (inputs, targets) in enumerate(loader):
                    inputs = inputs.to(self.device)
                    targets = targets.to(self.device)
                    pred = self.model(inputs)

                    # 정확도 계산을 위해 누적 전체 건수와 누적 전체 num_correct 건수 계산  
                    num_correct = (pred.argmax(-1) == targets).sum().item()
                    num_total += inputs.shape[0]
                    accu_num_correct += num_correct
                    eval_metric = accu_num_correct / num_total

                    progress_bar.update(1)
                    if batch_idx % 20 == 0 or (batch_idx + 1) == progress_bar.total:
                        progress_bar.set_postfix({"Accuracy": eval_metric})
        
        return eval_metric

    def predict_proba(self, inputs):
        self.model.eval()
        with torch.no_grad():
            inputs = inputs.to(self.device)
            outputs = self.model(inputs)
            #예측값을 반환하므로 targets은 필요 없음.
            #targets = targets.to(self.device)
            pred_proba = F.softmax(outputs, dim=-1) #또는 dim=1

        return pred_proba

    def predict(self, inputs):
        pred_proba = self.predict_proba(inputs)
        pred_class = torch.argmax(pred_proba, dim=-1)

        return pred_class

In [ ]:
trained_model = trainer.get_trained_model()

# 학습데이터와 동일하게 정규화된 데이터를 입력해야 함. 
# test_dataset = CIFAR10(root='./data', train=False, download=True, transform=ToTensor())
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)

predictor = Predictor(model=trained_model, device=device)
eval_metric = predictor.evaluate(test_loader)
print(f'test dataset evaluation:{eval_metric:.4f}')

In [ ]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
plt.figure(figsize=(1, 1))
plt.imshow(test_dataset.data[0])
plt.title(class_names[test_dataset.targets[0]])

print('target value:', test_dataset.targets[0])

In [ ]:
test_dataset[0]

In [ ]:
# 반드시 예측할 이미지는 tensor로, shape는 4차원으로 입력. 이를 위해 unsqueeze(0)
pred_class = predictor.predict(test_dataset[0][0].unsqueeze(0))
print('predicted class:', pred_class.item())

### 출력 Feature Map의 면적 계산하기
* 입력 Feature Map의 면적과 Convolution 적용 Kernel size, stride 및 padding에 따른 출력 Feature Map의 면적 계산
* I는 입력 Feature Map의 면적(크기), K는 Filter의 Kernel size, P는 Padding(정수), S는 Strides(정수)
* O = (I - K + 2P)/S + 1 

#### Stride가 1이고 Padding이 없는 경우 - Kernel size 3 적용
* I는 입력 Feature Map의 크기, K는 Filter의 Kernel size, P는 Padding(정수), S는 Strides(정수)
* O = (I - K + 2P)/1 + 1 = (5 - 3 + 0 )/1 + 1 = 3

In [ ]:
import torch.nn as nn
import torch

input = torch.randn(1, 5, 5)
conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=0) #kernel_size=5로 적용
output = conv1(input)
print(output.shape)

#### Stride가 1이고 Padding이 1인 경우
* O = (I - F + 2P)/S + 1 = (5 - 3 + 2 )/1 + 1 = 5

In [ ]:
input = torch.randn(1, 5, 5)
conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1) #padding='same'으로 적용 
output = conv1(input)
print(output.shape)

In [ ]:
# Zero Padding 이 적용된 Output 보기
input = torch.randn(1, 5, 5)
padding_layer = nn.ZeroPad2d(padding=1)
padded_input = padding_layer(input)
print('padded input shape:', padded_input.shape)

conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3)
output = conv1(padded_input)
print(output.shape)

#### Padding 시 상하 좌우가 다른 값을 넣기
* Conv2d의 padding값으로 Tuple을 입력. 예를 들어 padding=(1, 2)이면 상하가 1, 좌우가 2임.
* 상하가 다른값, 또는 좌우가 다른 값으로 Padding 적용하려면 nn.ZeroPad2d나 F.pad()함수를 사용해야 함.

In [ ]:
input = torch.randn(1, 5, 5)
conv1 = nn.Conv2d(in_channels=1, out_channels=16, 
                  kernel_size=3, padding=(1, 2)) # padding=Tuple에서 맨앞이 상하, 뒤가 좌우
output = conv1(input)
print(output.shape)

In [ ]:
# Zero Padding 이 적용된 Output 보기
input = torch.randn(1, 5, 5)
#padding = (left, right, top, bottom)로 좌,우,상,하 순서임.
padding_layer = nn.ZeroPad2d(padding=(0, 1, 0, 2))
padded_input = padding_layer(input)
print('padded input shape:', padded_input.shape)

conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3)
output = conv1(padded_input)
print(output.shape)

In [ ]:
import torch.nn.functional as F

input = torch.randn(1, 5, 5)
#pad = (left, right, top, bottom)로 좌,우,상,하 순서임.
padded_input = F.pad(input, pad=(0, 1, 0, 2), mode='constant', value=0)
print(padded_input.shape)

conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3)
output = conv1(padded_input)
print(output.shape)

#### Stride가 2이고 Padding이 없는 경우 - Kernel size 3 적용
* O = (I - K + 2P)/S + 1 = (5 - 3)/2 + 1 = 2

In [ ]:
input = torch.randn(1, 5, 5)
conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=2)
output = conv1(input)
print(output.shape)

#### Conv의 stride가 2이상일 경우, padding='same'은 적용할 수 없음. 

In [ ]:
input = torch.randn(1, 5, 5)
conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=2, padding='same')
output = conv1(input)
print(output.shape)

In [ ]:
input = torch.randn(1, 5, 5)
conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=2, padding=(1, 1)) 
output = conv1(input)
print(output.shape)

### 입력이 6X6에서 Kernel_size=3, Stride=2 적용
* O = (I - K + 2P)/2 + 1 = (6 - 3 + 0)/2 + 1 = 2.5 = 2
* stride=2 적용 시에는 맨 가장자리를 Convolution 적용하지 못하는 경우를 피하기 위해 일반적으로 padding 더해줌

In [ ]:
input = torch.randn(1, 6, 6)
conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=2) 
output = conv1(input)
print(output.shape)

In [ ]:
input = torch.randn(1, 6, 6)
conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=2, padding=(1, 1)) 
output = conv1(input)
print(output.shape)

### Maxpooling 적용

In [ ]:
input = torch.randn(1, 224, 224) # (1, 223, 223)
max_pool = nn.MaxPool2d(kernel_size=2, stride=2) 
output = max_pool(input)
print(output.shape)